In [11]:
import lyricsgenius as genius
import os
import json
from tqdm import tqdm
import matplotlib.pylab as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

# lol don't use my API key
geniusCreds = 'Lw6NjXtbU7NndUFHRCcOX9FdLhPzVokLIt9c4LWzsTxM10wF7EICGtWSSso8Ohsq'
# for more data, you can adjust the number of artists as well as the max number of songs
artist_names = [
    'James Taylor',
    'The Beatles',
    'Johnny Cash',
    'Van Morrison',
    'Bob Dylan',
#     'James Brown',
    'Aretha Franklin',
    'Fleetwood Mac',
    'J. Cole'
#     'Lil Wayne'
]

api = genius.Genius(geniusCreds)

Using TensorFlow backend.


In [5]:
artist = api.search_artist('J. Cole', max_songs=100)
artist.save_lyrics()

Searching for songs by J. Cole...

Song 1: "No Role Modelz"
Song 2: "Power Trip"
Song 3: "Wet Dreamz"
Song 4: "Crooked Smile"
Song 5: "Deja Vu"
Song 6: "MIDDLE CHILD"
Song 7: "Love Yourz"
Song 8: "G.O.M.D."
Song 9: "Apparently"
Song 10: "Fire Squad"
Song 11: "Kevin’s Heart"
Song 12: "KOD"
Song 13: "A Tale of 2 Citiez"
Song 14: "4 Your Eyez Only"
Song 15: "1985"
Song 16: "Neighbors"
Song 17: "She Knows"
Song 18: "Forbidden Fruit"
Song 19: "03' Adolescence"
Song 20: "Immortal"
Song 21: "Rich Niggaz"
Song 22: "Work Out"
Song 23: "ATM"
Song 24: "Change"
Song 25: "January 28th"
Song 26: "Nobody’s Perfect"
Song 27: "Let Nas Down"
Song 28: "False Prophets"
Song 29: "Villuminati"
Song 30: "She’s Mine, Pt. 1"
Song 31: "Born Sinner"
Song 32: "In the Morning"
Song 33: "Lost Ones"
Song 34: "​everybody dies"
Song 35: "Photograph"
Song 36: "Runaway"
Song 37: "Can’t Get Enough"
Song 38: "LAnd of the Snakes"
Song 39: "Sparks Will Fly"
Song 40: "FRIENDS"
Song 41: "Black Friday"
Song 42: "Trouble"
Song 

In [29]:
for artist_name in artist_names:
    artist = api.search_artist(artist_name, max_songs=100)
    artist.save_lyrics()

Searching for songs by Aretha Franklin...

Song 1: "Respect"
Song 2: "I Say A Little Prayer"
Song 3: "Think"


KeyboardInterrupt: 

In [4]:

for artist in tqdm(artist_names):
    a_mod = artist.replace(' ', '')
    filename = f'Lyrics_{a_mod}.json'
    with open(f'./data/{filename}') as f:
        data = json.load(f)
        song_obj = {}
        song_obj[artist] = []
        for song in data['songs']:
            song_obj[artist].append(song['lyrics'])
        with open(f'./data/cleaned/{artist}.json', 'w') as outfile:
            json.dump(song_obj, outfile)

100%|██████████| 8/8 [00:00<00:00, 43.67it/s]


### Neural Net Stuff

In [48]:
# get X and Y to train network

X = []
Y = []

index = 0
for filename in tqdm(os.listdir('./data/cleaned')):
    with open(f'./data/cleaned/{filename}') as file:
        data = json.load(file)[filename.replace('.json', '')]
        if len(data) == 100:
            for song in data:
                X.append(song)
                Y.append(index)
    index += 1

100%|██████████| 10/10 [00:00<00:00, 913.39it/s]


In [49]:
print(np.shape(X))
print(np.shape(Y))

(500,)
(500,)


In [50]:
big_word_list = {}
for song in X:
    for word in song.split(' '):
        if word not in big_word_list.keys():
            if (len(big_word_list.keys()) == 0):
                big_word_list[word] = 1
            else:
                big_word_list[word] = len(big_word_list) + 1

In [51]:
token_x = []
for song in X:
    new_song = []
    for word in song.split(' '):
        new_song.append(big_word_list[word])
    new_song = np.array(new_song)
    token_x.append(new_song)
X = token_x

In [52]:
cutoff = 100
index = 0
for song in X:
    if (len(song) < cutoff):
        Y.pop(index)
        X.pop(index)
    index += 1

print(len(X))
print(len(Y))

474
474


In [57]:
X.shape

(474,)

In [54]:
X = np.array(X)
Y = np.array(Y)

In [55]:
model = Sequential([
    Dense(128, input_shape=(474,)),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('softmax'),
    Dense(9)
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [56]:
model.fit(X, Y, epochs=5, batch_size=32)

ValueError: Error when checking input: expected dense_5_input to have shape (474,) but got array with shape (1,)